# Mining & Modelling Character Networks

In [8]:
import pandas as pd
import numpy as np
import networkx as nx
import io
import random

Paper Reference [Here](https://math.ryerson.ca/~abonato/papers/CharacterNetworks_WAW_Aug1_BDAEGH.pdf)

In [2]:
# constants
data_path = './data/data.csv'

## Mining Character Networks

```python
def main():
    filename = './twilightEdgesNames.csv'
    savename = './twilightEdgesIDs.txt'
    savename2 = './twilightEdgesIDsWeights.txt'
    
    E = pd.read_csv(filename)
    E1 = E['Source']
    E2 = E['Target']
    namesText = np.unique(np.vstack((E1,E2)))
    namesInds = [i for i in range(len(namesText))]
    # print namesText,namesInds
    E1 = E1.replace(namesText,namesInds)
    E2 = E2.replace(namesText,namesInds)
    #write to file
    out = np.column_stack((E1,E2))
    # labelNames = 'Source,Target'
    np.savetxt(savename,out,fmt=('%d','%d'),delimiter='\t',comments='')
    #save weights too
    np.savetxt(savename2,np.column_stack((out,E['weight'])),fmt=('%d','%d','%d'),delimiter='\t',comments='')

    print "n: %d" % len(namesText)
    # print "E: %d" % E['weight'].sum()
    print "E: %d" % E.shape[0]
```

## Create Network

In [5]:
twilight_names = [
    'Bella Swan', 'Edward Cullen', 'Jacob Black', 'Carlisle Cullen', 'Esme Cullen', 'Alice Cullen', 'Emmett Cullen',
    'Rosalie Hale', 'Jasper Hale', 'Renesmee Cullen', 'James', 'Victoria', 'Laurent', 'Riley Biers', 'Bree Tanner',
    'Sam Uley', 'Quil Ateara V', 'Embry Call', 'Paul Lahote', 'Jared Cameron', 'Leah Clearwater', 'Seth Clearwater',
    'Collin Littlesea', 'Brady Fuller', 'Charlie Swan', 'Renée Dwyer', 'Harry Clearwater', 'Billy Black', 'Tyler Crowley',
    'Lauren Mallory', 'Mike Newton', 'Jessica Stanley', 'Angela Weber', 'Eric Yorkie', 'Emily Young', 'Sue Clearwater',
    'Quil Ateara III', 'J. Jenks'
]

In [16]:
def sample_data(n, char_list = twilight_names):
    '''
    This function will generate random data
    '''
    d = pd.DataFrame(
        {
            'Source' : [random.choice(twilight_names) for _ in range(n)],
            'Target' : [random.choice(twilight_names) for _ in range(n)],
            'Weight' : [random.randint(1, 25) for _ in range(n)]
        }
    ).drop_duplicates()
    return d

In [17]:
E = sample_data(1000)

In [18]:
E1 = E['Source']
E2 = E['Target']

In [19]:
namesText = np.unique(np.vstack((E1,E2)))
namesInds = [i for i in range(len(namesText))]

In [20]:
# print namesText,namesInds
E1 = E1.replace(namesText,namesInds)
E2 = E2.replace(namesText,namesInds)

In [21]:
#write to file
out = np.column_stack((E1,E2))

In [24]:
# labelNames = 'Source,Target'
np.savetxt(savename,out,fmt=('%d','%d'),delimiter='\t',comments='')

In [25]:
#save weights too
np.savetxt(savename2,np.column_stack((out,E['weight'])),fmt=('%d','%d','%d'),delimiter='\t',comments='')

## k-Profile

# Modelling Character Networks

## Preferential Attachement

## Chung-Lu

## Binomial Random Graph

## Configuration Model

# Machine Learning